In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
! pip install nltk
! pip install spacy
! pip install beautifulsoup4
! pip install regex

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import nltk
import spacy
import re
from bs4 import BeautifulSoup
nltk.download("all")

In [ ]:
df = pd.read_csv('Tweets_01.csv', error_bad_lines=False, engine="python")

In [ ]:
df = df[['text','sentiment']]
df = df[0:1000]
df.head()

In [ ]:
#load the spacy model and functions related to stop words modification
spacy_model = spacy.load('en_core_web_sm')
from spacy.lang.en.stop_words import STOP_WORDS as spacy_stopwords
# print(spacy_model.Defaults.stop_words) #print the default stop words list
# spacy_model.Defaults.stop_words.remove("whatever") #remove single stop word from the list
# spacy_model.Defaults.stop_words -= {"whatever", "whenever"} #remove several stop words from the list
# spacy_model.Defaults.stop_words.add("my_new_stopword") #add single stop word in the default list
# spacy_model.stop_words |= {"my_new_stopword1","my_new_stopword2"} #add several stop words in the default list

In [ ]:
#most widely used shortforms are in the internet for social media data
contra_Expan_Dict = {
"ain`t": "am not","aren`t": "are not","can`t": "cannot","can`t`ve": "cannot have","`cause": "because",
"could`ve": "could have","couldn`t": "could not","couldn`t`ve": "could not have","didn`t": "did not",
"doesn`t": "does not","don`t": "do not","hadn`t": "had not","hadn`t`ve": "had not have","hasn`t": "has not",
"haven`t": "have not","he`d": "he would","he`d`ve": "he would have","he`ll": "he will","he`ll`ve": "he will have",
"he`s": "he is","how`d": "how did","how`d`y": "how do you","how`ll": "how will",
"how`s": "how does","i`d": "i would","i`d`ve": "i would have","i`ll": "i will","i`ll`ve": "i will have","i`m": "i am",
"i`ve": "i have","isn`t": "is not","it`d": "it would","it`d`ve": "it would have","it`ll": "it will","it`ll`ve": "it will have",
"it`s": "it is","let`s": "let us","ma`am": "madam","mayn`t": "may not","might`ve": "might have","mightn`t": "might not",
"mightn`t`ve": "might not have","must`ve": "must have","mustn`t": "must not","mustn`t`ve": "must not have","needn`t": "need not","needn`t`ve": "need not have",
"o`clock": "of the clock","oughtn`t": "ought not","oughtn`t`ve": "ought not have","shan`t": "shall not",
"sha`n`t": "shall not","shan`t`ve": "shall not have","she`d": "she would",
"she`d`ve": "she would have","she`ll": "she will","she`ll`ve": "she will have",
"she`s": "she is","should`ve": "should have","shouldn`t": "should not","shouldn`t`ve": "should not have","so`ve": "so have","so`s": "so is",
"that`d": "that would","that`d`ve": "that would have","that`s": "that is","there`d": "there would","there`d`ve": "there would have","there`s": "there is",
"they`d": "they would","they`d`ve": "they would have","they`ll": "they will","they`ll`ve": "they will have","they`re": "they are","they`ve": "they have",
"to`ve": "to have","wasn`t": "was not"," u ": " you "," ur ": " your "," n ": " and ","won`t": "would not",
"dis": "this","bak": "back","brng": "bring","sooo": "so", "afaik" :"as far as i know",
"afk" :"away from keyboard","asap" :"as soon as possible","atk" :"at the keyboard","atm" :"at the moment","a3" :"anytime, anywhere, anyplace",
"bak" :"back at keyboard","bbl" :"be back later","bbs" :"be back soon","bfn" :"bye for now","b4n" :"bye for now","brb" :"be right back",
"brt" :"be right there","btw" :"by the way","b4" :"before","b4n" :"bye for now","cu" :"see you","cul8r" :"see you later",
"cya" :"see you","faq" :"frequently asked questions","fc" :"fingers crossed","fwiw" :"for what it's worth","fyi" :"for your information",
"gal" :"get a life","gg" :"good game","gn" :"good night","gmta" :"great minds think alike","gr8" :"great!","g9" :"genius",
"ic" :"i see","icq" :"i seek you (also a chat program)","ilu" :"ilu: i love you","imho" :"in my honest/humble opinion","imo" :"in my opinion","iow" :"in other words",
"irl" :"in real life","kiss" :"keep it simple, stupid","ldr" :"long distance relationship","lmao" :"laugh my a.. off","lol" :"laughing out loud",
"ltns" :"long time no see","l8r" :"later","mte" :"my thoughts exactly","m8" :"mate","nrn" :"no reply necessary","oic" :"oh i see",
"pita" :"pain in the a..","prt" :"party","prw" :"parents are watching","qpsa?   que pasa?" :"","rofl" :"rolling on the floor laughing",
"roflol" :"rolling on the floor laughing out loud","rotflmao" :"rolling on the floor laughing my ass off","sk8" :"skate","stats" :"your sex and age",
"asl" :"age, sex, location","thx" :"thank you","ttfn" :"ta-ta for now!","ttyl" :"talk to you later",
"u" :"you","u2" :"you too","u4e" :"yours for ever","wb" :"welcome back","wtf" :"what the fuck","wtg" :"way to go!",
"wuf" :"where are you from?","w8" :"wait","7k" :"sick:-d laugher", "w/out": "without", "ihavent": "i have not"}

In [ ]:
#function for converting shortforms to it's expanded form based on contra_Expan_Dict
def expanded_form(x):
  if x in contra_Expan_Dict.keys():
    return(contra_Expan_Dict[x])
  else:
    return(x)

In [ ]:
#function for removing url punctuations and digits
def clean_with_re(x):
  x=str(x)
  x=re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'," ", x) #Remove URLs
  x=re.sub(r'[^\w ]+', "", x) # Remove Punctuation-1
  x=re.sub(r"[,!@&\'?\.$%_]"," ", x) # Remove Punctuation-2
  x=re.sub(r"\d+"," ", x) #Remove digits
  return(x)

In [ ]:
#function for removing HTML Tags
def remove_html(text):
    return BeautifulSoup(text, "lxml").text

In [ ]:
#function for counting the words
from collections import Counter
cnt = Counter()
for text in df["text"].values:
    for word in text.split():
        cnt[word] += 1

#function for removing the most frequent words
cnt.most_common(10)
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

#function for removing the most rare words
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

In [ ]:
#remove emojis
def remove_emoji(string):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [ ]:
#function for removing duplicate white spaces
def remove_duplicate_ws(x):
  x=str(x)
  x=" ".join(re.split("\s+", x, flags=re.UNICODE))
  return(x)

In [ ]:
def pre_processing(input_data, text_col):
  #convert all the input texts into lower case.
  input_data["text_col_clean"]=input_data[text_col].apply(lambda x:str(x).lower())
  #convert all the shortform of the input texts to its expanded form.
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:[expanded_form(t) for t in str(x).split()])
  #remove the stopwords based on spacy default package
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:[t for t in x if t not in spacy_stopwords])
  #remove the url, punctuations and digits from the input text
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:clean_with_re(x))
  #remove the HTML Tags from the input text
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:remove_html(x))
  #lemmatization - converting evary word to it's root form
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:" ".join([t.lemma_ for t in spacy_model(str(x))if t.lemma_ !="-PRON-" ]))
  #remove the most frequents words
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:remove_freqwords(x))
  #remove the most rare words
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:remove_rarewords(x))
  #remove the emojis
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:remove_emoji(x))
  #remove the duplicate whitespace.
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:remove_duplicate_ws(x))

In [ ]:
#call the pre_processing function
pre_processing(input_data=df, text_col="text")

In [ ]:
df.head()

In [ ]:
#check random text before and after pre-processing
print("Before Pre-processing: ",df["text"][300])
print("After Pre-processing: ",df["text_col_clean"][300])
#print the sentence length before and after pre-processing
print("Before Pre-processing: ",len(df["text"][300].split()))
print("After Pre-processing: ",len(df["text_col_clean"][300].split()))
#print the sentiment of the text
print("Sentiment of the text: ",df["sentiment"][300])